# SQLite3 con Python
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcReg69x-3oq5mXsx8JiUtq27ycIVHu32EaId2saTrschQ&s)

SQLite es una librería escrita en C, que implementa un motor de base de datos SQL completo, pequeño, rápido, autocontenido y de alta confiabilidad. SQLite es el motor de base de datos más utilizado a nivel mundial. SQLite esta disponible en todos los teléfonos móviles del mundo y muchas computadoras y se encuentra embebido en incontables aplicaciones de uso frecuente.

---

## Pasos para trabajar con una base de datos
1. Conectarse a una base de datos
2. Crear un cursor
3. Realizar alguna acción sobre la base de datos
   1. Crear una tabla (CREATE)
   1. Borrar una tabla (DROP)
   1. Insertar un registro (INSERT)
   1. Consultar información (SELECT, FROM, WHERE, *)
   1. Modificar información (UPDATE)
   1. Eliminar un registro (DELETE)
   1. Conformar la acción anterior (COMMIT)
   1. Deshacer las operaciones (ROLLBACK)
4. Cerrar la conexión con la base datos (CLOSE)

El motor de dabe de datos de SQLite3 esta incluído en todas las distribuciones de Python. La ventaja de SQLite3 es que es una base de datos basada en archivos, por lo que no existe un servidor de base de datos, sino solo un archivo que contiene toda la dB (sigla estándar de *database*). Se puede acceder también a realizar operaciones en la base de datos desde la consola de comandos con la instrucción:

    sqlite3
    
Esto retornará el prompt `sqlite>` donde se podrán introducír instrucciones SQL (solo hay que recordar colocar `;` al final de cada instrucción). Para conectarse a un archivo de base de datos en este modo, hay que ejecutar:

    sqlite> .open FILENAME
    
Para salir:

    sqlite> .quit

In [6]:
import sqlite3

### Crear una tabla
Para crear una tabla en un archivo de base de datos SQLite3, es necesario declarar el archivo y especificar el *esquema* de la tabla con la instrucción SQL `CREATE`; es decir, los nombres de las columnas de la tabla, así como los tipos de datos que contendrán. Estos tipos pueden ser:

* Enteros (INTEGER)
* Reales (REAL)
* Textos (TEXT)
* Vacio (NULL)
* Datos binarios grandes (BLOB)

El tipo de datos BOOLEAN no existe, y se almacena como INTEGER, ya sea 0 o 1.

Tampoco existe un tipo de datos para almacenar información de tiempo, por lo que se requerirá TEXT o INTEGER (para almacenar UNIX time).

In [24]:
# Se abre el archivo database.db que contiene la base datos
# Si el archivo no existe se creara un archivo nuevo
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos (opcional IF NOT EXISTS)
cur.execute("""CREATE TABLE IF NOT EXISTS pacientes (id INTEGER PRIMARY KEY,
                                                    nombre TEXT NOT NULL, 
                                                    peso INTEGER NOT NULL,
                                                    altura REAL NOT NULL)""")

# Se confirma la accion anterior
conn.commit()

# Se cierra la conexion con la base de datos
conn.close()

Si se quiere crear una tabla en la memoria de trabajo se puede utilizar la instrucción:

    conn = sqlite3.connect(":memory:")
    
Si se quiere trabajar con esta tabla volatil, recuerde no ejecutar `conn.close()` ya que estará cerrando la tabla y esto la eliminará de la memoria RAM.

### Eliminar una tabla
Para eliminar una tabla es necesario conectarse al archivo de base de datos y eliminar la tabla con la instrucción SQL `DROP`:

In [23]:
# Se abre el archivo database.db que contiene la base datos
# Si el archivo no existe se creara un archivo nuevo
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
# Un bloque try captura la excepcion en caso la tabla no exista
try:
    cur.execute("DROP TABLE pacientes")
except:
    print("ERROR: La tabla no existe")

# Se cierra la conexion con la base de datos
conn.close()

### Insertar informacion en una tabla
Para insertar información en una tabla se requiere llamar a la instrucción SQL `INSERT` con los datos organizados según la estructura de la tabla. Si se ha definido un `PRIMARY KEY`, este puede obviarse y el motor de la base de datos creará un valor para este campo único y consecutivo (incremental)

In [25]:
# Se inserta un valor a la tabla
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
cur.execute("INSERT INTO pacientes (nombre, peso, altura) VALUES (?, ?, ?)", ("Elvio Lado", 80, 1.72))

# Se confirma la accion anterior
conn.commit()

# Se cierra la conexion con la base de datos
conn.close()

In [29]:
# Se insertan multiples valores en una tabla segun los datos contenidos en una lista de tuplas.
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

personas = [("Elsa Payo", 54, 1.65),
            ("Armando Paredes", 88, 1.75),
            ("Susana Oria", 48, 1.60),
            ("Esteban Dido", 110, 1.68),
           ]

cur.executemany("INSERT INTO pacientes (nombre, peso, altura) VALUES (?, ?, ?)", personas)

# Se confirma la accion anterior
conn.commit()

# Se cierra la conexion con la base de datos
conn.close()

### Consultar una tabla
Para consultar una tabla, es necesario crear una consulta SQL con la instrucción `SELECT`:

In [46]:
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion la base de datos
# SELECT y fetchone() sobre el cursor
cur.execute("SELECT * FROM pacientes")
data = cur.fetchone()

# Se imprime el tipo de datos retornado y el valor
print(type(data))
print(data)

<class 'tuple'>
(1, 'Elvio Lado', 80, 1.72)


In [47]:
# Se ejecuta ua acción sobre la base de datos
# SELECT y fetchall() sobre el cursor
cur.execute("SELECT * FROM pacientes")
data = cur.fetchall()

# Se imprime el tipo de datos retornado y el valor
print(type(data))
print(data)

<class 'list'>
[(1, 'Elvio Lado', 80, 1.72), (2, 'Elsa Payo', 54, 1.65), (3, 'Armando Paredes', 88, 1.75), (4, 'Susana Oria', 48, 1.6), (5, 'Estaban Dido', 110, 1.68)]


In [48]:
# SELECT y cursor como iterable
cur.execute("SELECT * FROM pacientes")
for data in cur:
    print("ID: {:<3} Nombre: {:16} Peso: {:3} kg      Altura:{:.2f} m".format(*data))

# Se cierra la conexion con la base de datos
conn.close()

ID: 1   Nombre: Elvio Lado       Peso:  80 kg      Altura:1.72 m
ID: 2   Nombre: Elsa Payo        Peso:  54 kg      Altura:1.65 m
ID: 3   Nombre: Armando Paredes  Peso:  88 kg      Altura:1.75 m
ID: 4   Nombre: Susana Oria      Peso:  48 kg      Altura:1.60 m
ID: 5   Nombre: Estaban Dido     Peso: 110 kg      Altura:1.68 m


### Consultar una tabla con parametros de consulta
La instrucción SQL `SELECT` se puede combinar con otras instrucción SQL para obtener consultas complejas:

In [49]:
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
# Consulta con condiciones
cur.execute("SELECT * FROM pacientes WHERE peso < 60 AND altura > 1.60")
data = cur.fetchall()
print(data)

# Uso de LIKE para encontrar conicidencias
cur.execute("SELECT nombre, peso, altura FROM pacientes WHERE nombre LIKE '%el%'")
data = cur.fetchall()
print(data)

# Uso del comodin "?" para incluir campos externos
cur.execute("SELECT * FROM pacientes WHERE altura > ?", (1.70,))
data = cur.fetchall()
print(data)

# Se cierra la conexion con la base de datos
conn.close()

[(2, 'Elsa Payo', 54, 1.65)]
[('Elvio Lado', 80, 1.72), ('Elsa Payo', 54, 1.65)]
[(1, 'Elvio Lado', 80, 1.72), (3, 'Armando Paredes', 88, 1.75)]


### Actualización de un registro
Se puede actualizar los valor de un registro en la base de datos con la instrucción SQL `UPDATE`. Note que la instrucción `conn.commit()` es importante ya que de no estar presente, al cerrar la base de datos, la actualización no quedará registrada en el archivo de datos.

In [52]:
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
cur.execute("UPDATE pacientes SET peso = ? WHERE id = ?", (88, 1))

# Se confirma la accion anterior
conn.commit()

# Consulta
cur.execute("SELECT * FROM pacientes")
for data in cur:
    print("ID: {:<3} Nombre: {:16}   Peso: {:3}kg      Altura:{:.2f}m".format(*data))

# Se cierra la conexion con la base de datos
conn.close()

ID: 1   Nombre: Elvio Lado         Peso:  88kg      Altura:1.72m
ID: 2   Nombre: Elsa Payo          Peso:  54kg      Altura:1.65m
ID: 3   Nombre: Armando Paredes    Peso:  88kg      Altura:1.75m
ID: 4   Nombre: Susana Oria        Peso:  48kg      Altura:1.60m
ID: 5   Nombre: Estaban Dido       Peso: 110kg      Altura:1.68m


### Eliminar un registro
Para eliminar un registro de la base da datos, se utiliza la instrucción SQL `DELETE`. Al igual que en el caso anterior, se debe de confirmar la operación para que se haga efectiva en el archivo de datos.

In [53]:
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
cur.execute("DELETE FROM pacientes WHERE id = ?", (5,))

# Se confirma la accion anterior
conn.commit()

# Consulta
cur.execute("SELECT * FROM pacientes")
for data in cur:
    print("ID: {:<3} Nombre: {:16} Peso: {:3}kg      Altura:{:.2f}m".format(data[0], data[1], data[2], data[3]))

# Se cierra la conexion con la base de datos
conn.close()

ID: 1    Nombre: Elvio Lado       Peso:  88kg      Altura:1.72m
ID: 2    Nombre: Elsa Payo        Peso:  54kg      Altura:1.65m
ID: 3    Nombre: Armando Paredes  Peso:  88kg      Altura:1.75m
ID: 4    Nombre: Susana Oria      Peso:  48kg      Altura:1.60m


### Rollback
Las operaciones de base de datos pueden retroceder hasta un punto de inicio, deshaciendo todo lo hecho. A este proceso se le conoce como *rollback*. Este proceso deshace todas las operaciones hechas hasta el inicio de la conexión o hasta el último `commit`.

In [58]:
conn = sqlite3.connect("database.db")

# Se crea un cursor para poder acceder a la informacion
cur = conn.cursor()

# Se ejecuta una accion sobre la base de datos
print("------- SE ELIMINA REGISTRO ID 4 --------\n")
cur.execute("DELETE FROM pacientes WHERE id = ?", (4,))
cur.execute("UPDATE pacientes SET peso = ? WHERE id = ?", (0, 1))

# Consulta
cur.execute("SELECT * FROM pacientes")
for data in cur:
    print("ID: {:<3} Nombre: {:16} Peso: {:3}kg      Altura:{:.2f}m".format(*data))

print()
    
# Se deshace la ultima transaccion
conn.rollback()
print("------- ROLLBACK --------\n")

# Consulta
cur.execute("SELECT * FROM pacientes")
for data in cur:
    print("ID: {:<3} Nombre: {:16} Peso: {:3}kg      Altura:{:.2f}m".format(*data))

# Se cierra la conexion con la base de datos
conn.close()

------- SE ELIMINA REGISTRO ID 4 --------

ID: 1    Nombre: Elvio Lado       Peso:   0kg      Altura:1.72m
ID: 2    Nombre: Elsa Payo        Peso:  54kg      Altura:1.65m
ID: 3    Nombre: Armando Paredes  Peso:  88kg      Altura:1.75m

------- ROLLBACK --------

ID: 1    Nombre: Elvio Lado       Peso:  88kg      Altura:1.72m
ID: 2    Nombre: Elsa Payo        Peso:  54kg      Altura:1.65m
ID: 3    Nombre: Armando Paredes  Peso:  88kg      Altura:1.75m
ID: 4    Nombre: Susana Oria      Peso:  48kg      Altura:1.60m


### Bloque try... except... finally
Se puede utilizar un bloque `try` para poder controlar el `rollback` en caso de error, pero debe de considerarse que la base de datos debe de cerrarse se haya generado o no una excepción. Para estos casos, el bloque `try` puede contener la intrucción `finally` que contiene un bloque de instrucciones que se ejecutan se haya generado una excepción o no.

In [59]:
try:
    # Conectar a la base de datos
    conn = sqlite3.connect("database.db")
    cur = conn.cursor()
    
    # Ejecutar la accion sobre la base de datos
    # Esta inserción repite un PRIMARY KEY por lo que generará una excepción de Integridad en la base de datos
    cur.execute("INSERT INTO pacientes (id, nombre, peso, altura) VALUES (?, ?, ?, ?)", (1, "Dina Mita", 120, 1.55))
    
    # Confirmar la consulta (este commit no se ejecutará)
    conn.commit()
    
except sqlite3.IntegrityError:
    conn.rollback()
    print("ERROR: El 'id' ya existe en el registro. Los cambios no se guardan")
    
finally:
    # Cerrar la conexion incondicionalmente
    conn.close()

ERROR: El 'id' ya existe en el registro. Los cambios no se guardan


### Gestión de conexiones con un bloque `with` (Context Manager)
Las conexiones a una base de datos se pueden gestionar con la instrucción `with`, que controlará la confirmación de los cambios en el archivo de datos en caso sea necesario. Con la inclusión de un bloque `with` ya no será necesario ejecutar `conn.commit()`, **pero si de cerrar la conexión con `conn.close()`**.

In [60]:
conn = sqlite3.connect("database.db")

with conn:
    cur = conn.cursor()
    query = cur.execute("SELECT * FROM pacientes WHERE altura > 1.70")
    rows = query.fetchall()
    
    for data in rows:
        print(data)
    
conn.close()

(1, 'Elvio Lado', 88, 1.72)
(3, 'Armando Paredes', 88, 1.75)


## Clases para la gestion de una base de datos
Lo deseable es tener los diferentes aspectos de una aplicación en espacios separados y eso involucra a los detalles en la gestión de una base datos y la aplicación que utiliza la información. Esto obliga a que parte de una aplicación este escrita en un lenguaje de programación (en este caso, Python) y otra que contenga instrucciones en SQL. Lo ideal es que estos elementos se encuentren separados de forma tal que en el script principal se tenga uniformidad en el código. Esto se puede lograr construyendo una clase para la base de datos, para luego utilizarla en la aplicación.

In [105]:
class Database:
    datafile = "database.db"
    def __init__(self):
        self.conn = sqlite3.connect(Database.datafile)
        self.cur = self.conn.cursor()
        
    def __del__(self):
        self.conn.close()
        
    def nombres_pacientes(self):
        return self.cur.execute("SELECT nombre FROM pacientes ORDER BY nombre").fetchall()
    
    def pesos_pacientes(self, order='asc'):
        if order == 'asc':
            return self.cur.execute("SELECT nombre, peso FROM pacientes ORDER BY peso").fetchall()
        elif order == 'desc':
            return self.cur.execute("SELECT nombre, peso FROM pacientes ORDER BY peso DESC").fetchall()
        else:
            return [()]
        
    def pacientes_altura_promedio(self):
        return self.cur.execute("SELECT AVG(altura) FROM pacientes").fetchone()[0] 

Una vez construida la clase, el script de la aplicación contiene solamente código Python, lo que hace el código más légible y fácil de mantener.

In [112]:
db = Database()

print()
for idx, item in enumerate(db.nombres_pacientes(), start=1):
    print(f"{idx} - {item[0]}")

print()
for idx, item in enumerate(db.pesos_pacientes(), start=1):
    print(f"{idx} - {item[0]:20} {item[1]} kg")

print("\nAltura promedio:")
print(db.pacientes_altura_promedio(), "m")
    
del db


1 - Armando Paredes
2 - Elsa Payo
3 - Elvio Lado
4 - Susana Oria

1 - Susana Oria          48 kg
2 - Elsa Payo            54 kg
3 - Elvio Lado           88 kg
4 - Armando Paredes      88 kg

Altura promedio:
1.6800000000000002 m


## Programación por modelo de capas
En una programación por capas, cada clase conforma una "capa de abstracción" que modela un ambiente diferente. De esta forma, cada entorno tiene una clase diferentes que interactua con las demás.

Un ejemplo de esto es la interacción entre la clase Database y la clase App, de tal forma que una aplicación GUI en tkinter, pueda utilizar los recursos de una base de datos por medio de la clase Database.

In [17]:
import sqlite3
import tkinter as tk
import tkinter.ttk as ttk

class Database:
    datafile = "database.db"
    def __init__(self):
        self.conn = sqlite3.connect(Database.datafile)
        self.cur = self.conn.cursor()
        
    def __del__(self):
        self.conn.close()
        
    def nombres_pacientes(self):
        # Retorna una lista con los nombres de los pacientes (en orden alfabetico)
        data = self.cur.execute("SELECT nombre FROM pacientes ORDER BY nombre")
        return [item[0] for item in data]
    
    def data_pacientes(self, nombre):
        # Retorna una lista con la info de un paciente segun id: (peso, altura)
        return self.cur.execute("SELECT peso, altura FROM pacientes WHERE nombre = ?", (nombre,)).fetchone()
        

In [19]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("App dB")
        self.resizable(0, 0)
        
        self.dB = Database()
        
        self.var_nombre = tk.StringVar()
        self.var_peso = tk.StringVar()
        self.var_altura = tk.StringVar()
        
        frm = tk.Frame(self)
        frm1 = tk.Frame(frm)
        frm2 = tk.Frame(frm)
        
        frm.pack(padx=10, pady=10)
        frm1.pack(side=tk.LEFT, padx=10, pady=10, anchor=tk.N)
        frm2.pack(side=tk.LEFT, padx=10, pady=10)
        
        self.cboNombres = ttk.Combobox(frm1, value=self.dB.nombres_pacientes())
        self.cboNombres.pack(padx=5, pady=5)
        
        self.lblNombre = tk.Label(frm2, text="Nombre:")
        self.lblPeso = tk.Label(frm2, text="Peso:")
        self.lblAltura = tk.Label(frm2, text="Altura:")
        self.entNombre = tk.Entry(frm2, textvariable=self.var_nombre)
        self.entPeso = tk.Entry(frm2, textvariable=self.var_peso)
        self.entAltura = tk.Entry(frm2, textvariable=self.var_altura)
        
        self.lblNombre.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
        self.lblPeso.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
        self.lblAltura.grid(row=2, column=0, padx=5, pady=5, sticky=tk.E)
        self.entNombre.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        self.entPeso.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
        self.entAltura.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W)
        
        self.cboNombres.bind("<<ComboboxSelected>>", self.name_selected)
        
        
    def name_selected(self, event):
        name = self.cboNombres.get()
        peso, altura = self.dB.data_pacientes(name)
                
        self.var_nombre.set(name)
        self.var_peso.set(peso)
        self.var_altura.set(altura)
        
        
app = App().mainloop()